# P11 — La poule qui chante  
## Notebook 1 : Nettoyage, exploration et préparation des données

### Contexte
L’objectif du projet est d’identifier des **groupements de pays** pertinents pour une stratégie de développement à l’international (pré-sélection), à partir de données open data (FAO + éventuelles sources complémentaires)pour l'entreprise d'agro-alimentaire "La Poule qui Chante"

### Objectif de ce notebook
Ce notebook est **volontairement limité** à :
- l’import des fichiers sources ;
- le **nettoyage** (types, valeurs manquantes, doublons, libellés, cohérence) ;
- l’**analyse exploratoire** (EDA) ;
- les **jointures** et la construction d’un dataset final exploitable.

➡️ Les étapes d’**ACP et clustering** seront réalisées dans un **second notebook**, dédié uniquement à l’analyse et à la modélisation.

### Fichiers manipulés (dossier local)
Les fichiers sont stockés dans le même dossier que le notebook :  
`C:\Users\olivi\Documents\P11\Data`

Principaux fichiers attendus :
- `BilansAlimentaires_F_Toutes_les_Données_(Normalisé).csv` *(fichier volumineux)*
- `Population_F_Toutes_les_Données_(Normalisé).csv`
- `Statistiques-macro_Indicateurs_clés_F_Toutes_les_Données_(Normalisé).csv`
- `Données_de_la_sécurité_alimentaire_F_Toutes_les_Données_(Normalisé).csv`
- `PoliticalStability.csv`
- `DisponibiliteAlimentaire_2017.csv`
- `Population_2000_2018.csv`

Fichier optionnel (selon disponibilité et pertinence des variables) :
- `PoliticalStability.csv`

## 1. Import des bibliothèques
On importe les librairies nécessaires pour manipuler les données, visualiser rapidement et garder un notebook reproductible.


In [1]:
# Manipulation de données
import pandas as pd  # Pour manipuler les données (tableaux)
import numpy as np   # Pour les calculs mathématiques
from datetime import datetime  # Pour manipuler les dates
import pycountry

# Visualisation (EDA)
import matplotlib.pyplot as plt
import seaborn as sns

# Options d'affichage / confort
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

# Affichage pandas
pd.set_option('display.max_columns', None)  # Affiche toutes les colonnes
pd.set_option('display.max_rows', 100)      # Affiche jusqu'à 100 lignes
pd.set_option('display.float_format', '{:.2f}'.format)  # 2 décimales pour les nombres

# Configuration pour des graphiques plus jolis
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Bibliothèques importées avec succès !")


✅ Bibliothèques importées avec succès !


## 2. Paramétrage du projet et chemins de fichiers
On centralise le chemin du dossier `Data` pour éviter les chemins en dur partout dans le notebook et limiter les erreurs.


In [2]:
DATA_DIR = Path(r"C:\Users\olivi\Documents\P11\Data")

# (Optionnel) vérifier que le dossier existe
assert DATA_DIR.exists(), f"Dossier introuvable : {DATA_DIR}"
DATA_DIR


WindowsPath('C:/Users/olivi/Documents/P11/Data')

## 3. Chargement des fichiers sources
On charge les jeux de données du dossier `Data` pour pouvoir contrôler la structure (colonnes, types, volume) avant toute décision (garder/écarter, joindre, filtrer).


In [3]:
# --- Liste des fichiers attendus ---
FILES = {
    "fbs_norm": "BilansAlimentaires_F_Toutes_les_Données_(Normalisé).csv",  # très volumineux
    "pop_norm": "Population_F_Toutes_les_Données_(Normalisé).csv",
    "macro_norm": "Statistiques-macro_Indicateurs_clés_F_Toutes_les_Données_(Normalisé).csv",
    "foodsec_norm": "Données_de_la_sécurité_alimentaire_F_Toutes_les_Données_(Normalisé).csv",
    "dispo_2017": "DisponibiliteAlimentaire_2017.csv",
    "pop_2000_2018": "Population_2000_2018.csv",
    "pol_stab": "PoliticalStability.csv",  # optionnel
}

paths = {k: DATA_DIR / v for k, v in FILES.items()}

missing = [k for k, p in paths.items() if not p.exists()]
if missing:
    print("⚠️ Fichiers manquants :", missing)
else:
    print("✅ Tous les fichiers listés sont présents.")
    

# --- Détection simple du séparateur (',' ou ';') ---
def sniff_sep(path, default=","):
    with open(path, "r", encoding="utf-8", errors="replace") as f:
        first_line = f.readline()
    return ";" if first_line.count(";") > first_line.count(",") else default

def read_fao_csv(path, nrows=None):
    """
    Lecture robuste :
    - essaie d'abord engine='c' (rapide, compatible low_memory)
    - fallback engine='python' si le parsing est exotique (mais sans low_memory)
    """
    sep = sniff_sep(path)
    common_kwargs = dict(
        sep=sep,
        encoding="utf-8",
        encoding_errors="replace",
        nrows=nrows,
        on_bad_lines="skip"   # si quelques lignes sont bancales
    )

    try:
        return pd.read_csv(path, engine="c", low_memory=False, **common_kwargs)
    except Exception as e_c:
        print(f"⚠️ Lecture engine='c' échouée pour {path.name} → fallback engine='python' ({type(e_c).__name__})")
        return pd.read_csv(path, engine="python", **common_kwargs)

# --- Rechargement des datasets ---
LOAD_FULL_FBS = False
FBS_SAMPLE_NROWS = 200_000

dt_fbs = read_fao_csv(paths["fbs_norm"], nrows=None if LOAD_FULL_FBS else FBS_SAMPLE_NROWS)
dt_pop_norm = read_fao_csv(paths["pop_norm"])
dt_macro_norm = read_fao_csv(paths["macro_norm"])
dt_foodsec_norm = read_fao_csv(paths["foodsec_norm"])
dt_dispo_2017 = read_fao_csv(paths["dispo_2017"])
dt_pop_2000_2018 = read_fao_csv(paths["pop_2000_2018"])

dt_pol_stab = None
if paths["pol_stab"].exists():
    dt_pol_stab = read_fao_csv(paths["pol_stab"])

datasets = {
    "dt_fbs": dt_fbs,
    "dt_pop_norm": dt_pop_norm,
    "dt_macro_norm": dt_macro_norm,
    "dt_foodsec_norm": dt_foodsec_norm,
    "dt_dispo_2017": dt_dispo_2017,
    "dt_pop_2000_2018": dt_pop_2000_2018,
}
if dt_pol_stab is not None:
    datasets["dt_pol_stab"] = dt_pol_stab

for name, df in datasets.items():
    print(f"{name:18s} | shape={df.shape} | colonnes={len(df.columns)}")


✅ Tous les fichiers listés sont présents.
dt_fbs             | shape=(200000, 14) | colonnes=14
dt_pop_norm        | shape=(168404, 13) | colonnes=13
dt_macro_norm      | shape=(708973, 13) | colonnes=13
dt_foodsec_norm    | shape=(278940, 13) | colonnes=13
dt_dispo_2017      | shape=(176600, 14) | colonnes=14
dt_pop_2000_2018   | shape=(4411, 15) | colonnes=15
dt_pol_stab        | shape=(3526, 4) | colonnes=4


## 4. Exploration initiale — `dt_fbs` (Bilans alimentaires)
On commence par le fichier principal car c’est lui qui conditionne la structure globale (colonnes, encodage, types) et les futures jointures.


In [4]:
name = "dt_fbs"
df = dt_fbs

print(f"{name} | shape={df.shape}")
print("\nColonnes :")
print(list(df.columns))

print("\nAperçu (10 premières lignes) :")
display(df.head(10))

print("\nAperçu (10 dernières lignes) :")
display(df.tail(10))


dt_fbs | shape=(200000, 14)

Colonnes :
['Code zone', 'Code zone (M49)', 'Zone', 'Code Produit', 'Code Produit (FBS)', 'Produit', 'Code Élément', 'Élément', 'Code année', 'Année', 'Unité', 'Valeur', 'Symbole', 'Note']

Aperçu (10 premières lignes) :


,Code zone,Code zone (M49),Zone,Code Produit,Code Produit (FBS),Produit,Code Élément,Élément,Code année,Année,Unité,Valeur,Symbole,Note
0,2,'004,Afghanistan,2501,'S2501,Population,511,Population totale,2010,2010,1000 No,28284.09,X,NaN
1,2,'004,Afghanistan,2501,'S2501,Population,511,Population totale,2011,2011,1000 No,29347.71,X,NaN
2,202,'710,Afrique du Sud,2501,'S2501,Population,511,Population totale,2010,2010,1000 No,52344.05,X,NaN
3,202,'710,Afrique du Sud,2501,'S2501,Population,511,Population totale,2011,2011,1000 No,52995.21,X,NaN
4,3,'008,Albanie,2501,'S2501,Population,511,Population totale,2010,2010,1000 No,2928.72,X,NaN
5,2,'004,Afghanistan,2501,'S2501,Population,511,Population totale,2012,2012,1000 No,30560.03,X,NaN
6,4,'012,Algérie,2501,'S2501,Population,511,Population totale,2010,2010,1000 No,36188.24,X,NaN
7,2,'004,Afghanistan,2501,'S2501,Population,511,Population totale,2013,2013,1000 No,31622.70,X,NaN
8,3,'008,Albanie,2501,'S2501,Population,511,Population totale,2011,2011,1000 No,2911.50,X,NaN
9,202,'710,Afrique du Sud,2501,'S2501,Population,511,Population totale,2012,2012,1000 No,53782.57,X,NaN



Aperçu (10 dernières lignes) :


,Code zone,Code zone (M49),Zone,Code Produit,Code Produit (FBS),Produit,Code Élément,Élément,Code année,Année,Unité,Valeur,Symbole,Note
199990,143,'504,Maroc,2807,'S2807,Riz et produits,645,Disponibilité alimentaire en quantité (kg/pers...,2013,2013,kg/personne,1.57,E,NaN
199991,129,'450,Madagascar,2807,'S2807,Riz et produits,5142,Nourriture,2010,2010,1000 t,3392.00,I,NaN
199992,137,'480,Maurice,2807,'S2807,Riz et produits,645,Disponibilité alimentaire en quantité (kg/pers...,2012,2012,kg/personne,85.99,E,NaN
199993,256,'442,Luxembourg,2807,'S2807,Riz et produits,661,Disponibilité alimentaire (Kcal),2021,2021,millions de kcal,12405.67,I,NaN
199994,123,'430,Libéria,2807,'S2807,Riz et produits,681,Disponibilité de matière grasse en quantité (t),2013,2013,tonnes,6051.62,I,NaN
199995,121,'422,Liban,2807,'S2807,Riz et produits,671,Disponibilité de protéines en quantité (t),2023,2023,tonnes,3854.75,I,NaN
199996,126,'440,Lituanie,2807,'S2807,Riz et produits,661,Disponibilité alimentaire (Kcal),2017,2017,millions de kcal,30937.12,I,NaN
199997,119,'428,Lettonie,2807,'S2807,Riz et produits,661,Disponibilité alimentaire (Kcal),2023,2023,millions de kcal,18806.65,I,NaN
199998,113,'417,Kirghizistan,2807,'S2807,Riz et produits,5142,Nourriture,2020,2020,1000 t,53.00,I,NaN
199999,124,'434,Libye,2807,'S2807,Riz et produits,671,Disponibilité de protéines en quantité (t),2017,2017,tonnes,11401.55,I,NaN


## 5. Structure et types — `dt_fbs`
On vérifie les types de données et la présence de valeurs manquantes pour identifier ce qu’il faudra convertir/nettoyer (ex : `Valeur`, `Année`, codes M49).


In [5]:
display(df.info())

# % de valeurs manquantes par colonne (top 20)
na_pct = (df.isna().mean() * 100).sort_values(ascending=False)
display(na_pct[na_pct > 0].head(20).to_frame("NA_%"))

# Doublons
print("Doublons (lignes identiques) :", df.duplicated().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Code zone           200000 non-null  int64  
 1   Code zone (M49)     200000 non-null  object 
 2   Zone                200000 non-null  object 
 3   Code Produit        200000 non-null  int64  
 4   Code Produit (FBS)  200000 non-null  object 
 5   Produit             200000 non-null  object 
 6   Code Élément        200000 non-null  int64  
 7   Élément             200000 non-null  object 
 8   Code année          200000 non-null  int64  
 9   Année               200000 non-null  int64  
 10  Unité               200000 non-null  object 
 11  Valeur              200000 non-null  float64
 12  Symbole             200000 non-null  object 
 13  Note                0 non-null       float64
dtypes: float64(2), int64(5), object(7)
memory usage: 21.4+ MB


None

,NA_%
Note,100.00


Doublons (lignes identiques) : 0


## 6. Contrôle des noms de colonnes — `dt_fbs`
On repère les problèmes classiques : espaces en trop, caractères invisibles, accents cassés (mojibake) et doublons de colonnes après nettoyage.


In [6]:
import re

cols = pd.Series(df.columns, name="original")
col_audit = pd.DataFrame({
    "original": cols,
    "strip": cols.str.strip(),
    "leading/trailing_spaces": cols.ne(cols.str.strip()),
    "double_spaces": cols.str.contains(r"\s{2,}", regex=True),
    "nbsp": cols.str.contains("\u00A0", regex=False),
    "mojibake": cols.str.contains("Ã|Â|�", regex=True),
})

col_audit["proposition"] = (
    col_audit["strip"]
    .str.replace("\u00A0", " ", regex=False)
    .str.replace(r"\s{2,}", " ", regex=True)
)

print("Colonnes dupliquées après strip ?",
      col_audit["strip"].duplicated().any())
display(col_audit)


Colonnes dupliquées après strip ? False


,original,strip,leading/trailing_spaces,double_spaces,nbsp,mojibake,proposition
0,Code zone,Code zone,False,False,False,False,Code zone
1,Code zone (M49),Code zone (M49),False,False,False,False,Code zone (M49)
2,Zone,Zone,False,False,False,False,Zone
3,Code Produit,Code Produit,False,False,False,False,Code Produit
4,Code Produit (FBS),Code Produit (FBS),False,False,False,False,Code Produit (FBS)
5,Produit,Produit,False,False,False,False,Produit
6,Code Élément,Code Élément,False,False,False,False,Code Élément
7,Élément,Élément,False,False,False,False,Élément
8,Code année,Code année,False,False,False,False,Code année
9,Année,Année,False,False,False,False,Année


## 7. Vérification rapide des colonnes “clés” — `dt_fbs`
On vérifie que les colonnes attendues sont présentes (zones, produits, éléments, année, valeur) et on inspecte quelques valeurs uniques pour repérer les guillemets/apostrophes parasites.


In [7]:
expected_cols = [
    "Code zone", "Code zone (M49)", "Zone",
    "Code Produit", "Code Produit (FBS)", "Produit",
    "Code Élément", "Élément",
    "Code année", "Année",
    "Unité", "Valeur", "Symbole", "Note"
]

missing_cols = [c for c in expected_cols if c not in df.columns]
print("Colonnes attendues manquantes :", missing_cols)

# Mini inspection des valeurs (top 10) sur quelques colonnes critiques
for c in ["Code zone (M49)", "Zone", "Produit", "Élément", "Année", "Unité"]:
    if c in df.columns:
        print(f"\n--- {c} (exemples) ---")
        display(df[c].astype(str).head(10))
        print("Nb valeurs uniques (sur l'échantillon chargé) :", df[c].nunique(dropna=True))


Colonnes attendues manquantes : []

--- Code zone (M49) (exemples) ---


0    '004
1    '004
2    '710
3    '710
4    '008
5    '004
6    '012
7    '004
8    '008
9    '710
Name: Code zone (M49), dtype: object

Nb valeurs uniques (sur l'échantillon chargé) : 213

--- Zone (exemples) ---


0       Afghanistan
1       Afghanistan
2    Afrique du Sud
3    Afrique du Sud
4           Albanie
5       Afghanistan
6           Algérie
7       Afghanistan
8           Albanie
9    Afrique du Sud
Name: Zone, dtype: object

Nb valeurs uniques (sur l'échantillon chargé) : 213

--- Produit (exemples) ---


...

Nb valeurs uniques (sur l'échantillon chargé) : 21

--- Année (exemples) ---


0    2010
1    2011
2    2010
3    2011
4    2010
5    2012
6    2010
7    2013
8    2011
9    2012
Name: Année, dtype: object

Nb valeurs uniques (sur l'échantillon chargé) : 14

--- Unité (exemples) ---


0    1000 No
1    1000 No
2    1000 No
3    1000 No
4    1000 No
5    1000 No
6    1000 No
7    1000 No
8    1000 No
9    1000 No
Name: Unité, dtype: object

Nb valeurs uniques (sur l'échantillon chargé) : 7


## 8. Colonnes numériques potentielles — `dt_fbs`
On identifie ce qui devrait être numérique mais est importé en texte (fréquent sur `Valeur` et certains codes), pour préparer la conversion propre.


In [8]:
# Dtypes
display(df.dtypes.to_frame("dtype"))

# Test “numérique-like” sur Valeur si elle est en object
if "Valeur" in df.columns and df["Valeur"].dtype == "object":
    s = df["Valeur"].dropna().astype(str).head(2000)
    cleaned = (s.str.replace('"', '', regex=False)
                 .str.replace("'", "", regex=False)
                 .str.replace(",", ".", regex=False)
                 .str.strip())
    ratio_numeric = cleaned.str.match(r"^-?\d+(\.\d+)?$").mean()
    print(f"Valeur importée en texte → ratio numeric-like (sur 2000 valeurs) : {ratio_numeric:.3f}")
    print("Exemples bruts :", s.head(10).tolist())
    print("Exemples nettoyés :", cleaned.head(10).tolist())


,dtype
Code zone,int64
Code zone (M49),object
Zone,object
Code Produit,int64
Code Produit (FBS),object
Produit,object
Code Élément,int64
Élément,object
Code année,int64
Année,int64


In [9]:
# Voir tous les types d'indicateurs
dt_fbs['Élément'].unique()

# Compter combien de lignes par type d'indicateur
dt_fbs['Élément'].value_counts()

Élément
Disponibilité alimentaire (Kcal/personne/jour)                   16299
Disponibilité alimentaire (Kcal)                                 16147
Disponibilité de protéines en quantité (g/personne/jour)         15939
Disponibilité de protéines en quantité (t)                       15804
Disponibilité de matière grasse en quantité (g/personne/jour)    15584
Disponibilité de matière grasse en quantité (t)                  15426
Importations - quantité                                           9200
Variation de stock                                                8895
Disponibilité intérieure                                          8782
Résidus                                                           8259
Aliments pour animaux                                             8173
Exportations - quantité                                           8058
Nourriture                                                        8045
Disponibilité alimentaire en quantité (kg/personne/an)            779

## 9. Lecture complète de `dt_fbs` en streaming (chunks)
On parcourt tout le fichier sans le charger en mémoire pour lister les modalités (`Produit`, `Élément`, `Unité`) et préparer les filtres avant le pivot.


In [10]:
from collections import Counter

# Colonnes utiles (on évite Note, souvent vide, et on garde le strict nécessaire)
FBS_USECOLS = [
    "Code zone", "Code zone (M49)", "Zone",
    "Code Produit", "Code Produit (FBS)", "Produit",
    "Code Élément", "Élément",
    "Code année", "Année",
    "Unité", "Valeur",
    "Symbole"  # optionnel, mais utile si on veut repérer des flags
]

# dtypes pour limiter la mémoire (ajuste si besoin)
FBS_DTYPES = {
    "Code zone": "int32",
    "Code Produit": "int32",
    "Code Élément": "int32",
    "Code année": "int32",
    "Année": "int32",
    "Valeur": "float64",  # garde float64 pour éviter surprises
    # les codes texte restent en object
}

fbs_path = paths["fbs_norm"]
fbs_sep = sniff_sep(fbs_path)

CHUNKSIZE = 500_000  # augmente/diminue selon ta RAM

prod_counter = Counter()
elem_counter = Counter()
unit_counter = Counter()

for i, chunk in enumerate(pd.read_csv(
    fbs_path,
    sep=fbs_sep,
    usecols=FBS_USECOLS,
    dtype=FBS_DTYPES,
    encoding="utf-8",
    encoding_errors="replace",
    engine="c",
    low_memory=False,
    chunksize=CHUNKSIZE
)):
    # comptes de modalités (sur tout le fichier)
    prod_counter.update(chunk["Produit"].dropna().astype(str).str.strip().tolist())
    elem_counter.update(chunk["Élément"].dropna().astype(str).str.strip().tolist())
    unit_counter.update(chunk["Unité"].dropna().astype(str).str.strip().tolist())

    if (i + 1) % 5 == 0:
        print(f"Chunks lus: {i+1} (≈ {(i+1)*CHUNKSIZE:,} lignes)")

print("✅ Scan terminé.")
print("Nb Produits uniques :", len(prod_counter))
print("Nb Éléments uniques :", len(elem_counter))
print("Nb Unités uniques   :", len(unit_counter))


Chunks lus: 5 (≈ 2,500,000 lignes)
Chunks lus: 10 (≈ 5,000,000 lignes)
✅ Scan terminé.
Nb Produits uniques : 119
Nb Éléments uniques : 21
Nb Unités uniques   : 7


## 10. Modalités principales de `Produit`, `Élément`, `Unité`
On affiche les valeurs les plus fréquentes pour identifier rapidement les produits et éléments pertinents à conserver.


In [11]:
# Top modalités (par fréquence)
dt_prod_top = pd.DataFrame(prod_counter.most_common(40), columns=["Produit", "nb_lignes"])
dt_elem_top = pd.DataFrame(elem_counter.most_common(60), columns=["Élément", "nb_lignes"])
dt_unit_top = pd.DataFrame(unit_counter.most_common(20), columns=["Unité", "nb_lignes"])

print("Top Produits :")
display(dt_prod_top)

print("Top Éléments :")
display(dt_elem_top)

print("Top Unités :")
display(dt_unit_top)


Top Produits :


,Produit,nb_lignes
0,Lait - Excl Beurre,101130
1,Oeufs,95386
2,Boissons Alcooliques,85976
3,Miscellanees,75028
4,Cèrèales - Excl Bière,55233
5,Cultures Oleágineuses,54617
6,Racines Amyl,53183
7,Maïs et produits,52869
8,Blé et produits,51670
9,Pommes de Terre et produits,51667


Top Éléments :


,Élément,nb_lignes
0,Disponibilité intérieure,329755
1,Importations - quantité,318901
2,Disponibilité de matière grasse en quantité (g...,310269
3,Disponibilité de matière grasse en quantité (t),310269
4,Disponibilité alimentaire (Kcal/personne/jour),310263
5,Disponibilité alimentaire (Kcal),310263
6,Disponibilité de protéines en quantité (g/pers...,310263
7,Disponibilité de protéines en quantité (t),310263
8,Nourriture,308034
9,Disponibilité alimentaire en quantité (kg/pers...,308034


Top Unités :


,Unité,nb_lignes
0,1000 t,2647972
1,g/personne/jour,620532
2,tonnes,620532
3,kcal/personne/jour,310263
4,millions de kcal,310263
5,kg/personne,308034
6,1000 No,2901


## 11. Liste complète des `Élément` (triée) pour décider quoi garder
On extrait la liste exhaustive et triée des éléments afin de sélectionner ceux qui serviront au pivot (indicateurs).


In [12]:
dt_elements = pd.DataFrame(sorted(elem_counter.keys()), columns=["Élément"])
display(dt_elements)


,Élément
0,Alimentation pour touristes
1,Aliments pour animaux
2,Autres utilisations (non alimentaire)
3,Disponibilité alimentaire (Kcal)
4,Disponibilité alimentaire (Kcal/personne/jour)
5,Disponibilité alimentaire en quantité (kg/pers...
6,Disponibilité de matière grasse en quantité (g...
7,Disponibilité de matière grasse en quantité (t)
8,Disponibilité de protéines en quantité (g/pers...
9,Disponibilité de protéines en quantité (t)


In [13]:
dt_produit = pd.DataFrame(sorted(prod_counter.keys()), columns=["Produit"])
with pd.option_context('display.max_rows', None):
    display(dt_produit)

,Produit
0,Abats
1,Abats Comestible
2,"Agrumes, Autres"
3,"Alcool, non Comestible"
4,Aliments pour enfants
5,Ananas et produits
6,Animaux Aquatiques Autre
7,Arachides
8,Avoine
9,Bananes


## 12. Paramètres de filtrage
On définit les filtres (année(s), produit(s), éléments) pour recharger le fichier complet mais en version réduite, avant le pivot.


In [14]:
# ✅ À ajuster si besoin
YEARS_KEEP = list(range(2010, 2018))

PRODUCTS_KEEP = [
    # MARCHÉ DIRECT (2 produits)
    'Viande de Volailles',  # ← CRITIQUE
    'Oeufs',
    
    # CONCURRENTS (5 produits)
    'Viande de Bovins',
    'Viande de porcins',
    'Viande d\'Ovins/Caprins',
    'Viande, Autre',
    'Poisson & Fruits de Mer',
    
    # CONTEXTE (4 produits)
    'Viande',  # Agrégat
    'Lait - Excl Beurre',
    'Légumineuses Sèches',
    'Produits Animaux',
    'Produits Vegetaux',
]

ELEMENTS_KEEP = [
    'Population totale',
    'Production',
    'Importations - quantité',
    'Exportations - quantité',
    'Disponibilité alimentaire en quantité (kg/personne/jour)',
    'Disponibilité de protéines en quantité (g/personne/jour)',
    'Nourriture'
]

print("YEARS_KEEP    :", YEARS_KEEP)
print("PRODUCTS_KEEP :", PRODUCTS_KEEP)
print("ELEMENTS_KEEP :", ELEMENTS_KEEP)


YEARS_KEEP    : [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
PRODUCTS_KEEP : ['Viande de Volailles', 'Oeufs', 'Viande de Bovins', 'Viande de porcins', "Viande d'Ovins/Caprins", 'Viande, Autre', 'Poisson & Fruits de Mer', 'Viande', 'Lait - Excl Beurre', 'Légumineuses Sèches', 'Produits Animaux', 'Produits Vegetaux']
ELEMENTS_KEEP : ['Population totale', 'Production', 'Importations - quantité', 'Exportations - quantité', 'Disponibilité alimentaire en quantité (kg/personne/jour)', 'Disponibilité de protéines en quantité (g/personne/jour)', 'Nourriture']


## 13. Rechargement complet filtré de `dt_fbs` (dataset réduit)
On relit le fichier en streaming mais on ne conserve que les lignes correspondant aux filtres choisis, pour obtenir un dataset manipulable.


In [15]:
# ---- Paramètres
YEARS_KEEP = list(range(2010, 2018))

PRODUCTS_KEEP = [
    "Viande de Volailles",
    "Viande",
]

ELEMENTS_KEEP = [
    "Population totale", 
    "Production",
    "Importations - quantité",
    "Exportations - quantité",
    "Disponibilité de protéines en quantité (g/personne/jour)",
]

# ---- Lecture filtrée en chunks
filtered_chunks = []

for chunk in pd.read_csv(
    fbs_path,
    sep=fbs_sep,
    usecols=FBS_USECOLS,
    dtype=FBS_DTYPES,
    encoding="utf-8",
    encoding_errors="replace",
    engine="c",
    low_memory=False,
    chunksize=CHUNKSIZE,
):
    # nettoyage minimal sur champs texte
    for c in ["Zone", "Produit", "Élément", "Unité", "Code zone (M49)", "Code Produit (FBS)"]:
        if c in chunk.columns:
            chunk[c] = chunk[c].astype(str).str.strip()

   # filtres
    m_year = chunk["Année"].isin(YEARS_KEEP)
    m_prod = chunk["Produit"].isin(PRODUCTS_KEEP)
    m_elem = chunk["Élément"].isin(ELEMENTS_KEEP)

    sub = chunk[m_year & m_prod & m_elem].copy()
    if not sub.empty:
        filtered_chunks.append(sub)

dt_fbs_small = pd.concat(filtered_chunks, ignore_index=True) if filtered_chunks else pd.DataFrame(columns=FBS_USECOLS)

print("dt_fbs_small | shape =", dt_fbs_small.shape)
display(dt_fbs_small.head(10))

dt_fbs_small | shape = (12791, 13)


,Code zone,Code zone (M49),Zone,Code Produit,Code Produit (FBS),Produit,Code Élément,Élément,Code année,Année,Unité,Valeur,Symbole
0,116,'408,République populaire démocratique de Corée,2943,'S2943,Viande,5511,Production,2010,2010,1000 t,324.00,E
1,116,'408,République populaire démocratique de Corée,2943,'S2943,Viande,5511,Production,2011,2011,1000 t,340.00,E
2,116,'408,République populaire démocratique de Corée,2943,'S2943,Viande,5511,Production,2012,2012,1000 t,343.00,E
3,116,'408,République populaire démocratique de Corée,2943,'S2943,Viande,5511,Production,2013,2013,1000 t,343.00,E
4,116,'408,République populaire démocratique de Corée,2943,'S2943,Viande,5511,Production,2014,2014,1000 t,339.00,E
5,116,'408,République populaire démocratique de Corée,2943,'S2943,Viande,5511,Production,2015,2015,1000 t,345.00,E
6,116,'408,République populaire démocratique de Corée,2943,'S2943,Viande,5511,Production,2016,2016,1000 t,354.00,E
7,116,'408,République populaire démocratique de Corée,2943,'S2943,Viande,5511,Production,2017,2017,1000 t,356.00,E
8,116,'408,République populaire démocratique de Corée,2943,'S2943,Viande,5611,Importations - quantité,2010,2010,1000 t,2.00,E
9,116,'408,République populaire démocratique de Corée,2943,'S2943,Viande,5611,Importations - quantité,2011,2011,1000 t,2.00,E


## 14. Contrôles avant pivot : unités et doublons
On contrôle l’unicité des combinaisons (pays, année, produit, élément, unité) et on vérifie les unités utilisées pour chaque élément afin d’éviter un pivot incohérent.


In [16]:
# 1) unités par élément
unit_check = (
    dt_fbs_small
    .groupby("Élément")["Unité"]
    .nunique()
    .sort_values(ascending=False)
    .to_frame("nb_unités")
)
display(unit_check)

# afficher quelles unités pour les éléments problématiques (si nb_unités > 1)
multi_units = unit_check.query("nb_unités > 1").index.tolist()
if multi_units:
    print("⚠️ Éléments avec plusieurs unités (à trancher avant pivot) :", multi_units)
    display(
        dt_fbs_small[dt_fbs_small["Élément"].isin(multi_units)]
        .groupby("Élément")["Unité"]
        .unique()
        .to_frame("unités")
    )
else:
    print("✅ Chaque élément n'utilise qu'une seule unité (sur le périmètre filtré).")

# 2) doublons sur clés de pivot (sans unité)
pivot_keys = ["Code zone (M49)", "Zone", "Année", "Produit", "Élément"]
dup_n = dt_fbs_small.duplicated(subset=pivot_keys).sum()
print("Doublons sur (pays, année, produit, élément) :", dup_n)

if dup_n > 0:
    display(
        dt_fbs_small[dt_fbs_small.duplicated(subset=pivot_keys, keep=False)]
        .sort_values(pivot_keys)
        .head(50)
    )


,nb_unités
Élément,
Disponibilité de protéines en quantité (g/personne/jour),1
Exportations - quantité,1
Importations - quantité,1
Production,1


✅ Chaque élément n'utilise qu'une seule unité (sur le périmètre filtré).
Doublons sur (pays, année, produit, élément) : 0


In [17]:
present_elements = set(dt_fbs_small["Élément"].unique())
missing_elements = [e for e in ELEMENTS_KEEP if e not in present_elements]

print("Éléments présents :", sorted(present_elements))
print("\nÉléments manquants vs ELEMENTS_KEEP :", missing_elements)


Éléments présents : ['Disponibilité de protéines en quantité (g/personne/jour)', 'Exportations - quantité', 'Importations - quantité', 'Production']

Éléments manquants vs ELEMENTS_KEEP : ['Population totale']


## 15. Résolution des doublons de clés : choix d’un code canonique par Produit (et par M49 si besoin)
On évite les doublons dus à plusieurs codes pour un même libellé en conservant le code le plus fréquent, puis on supprime les doublons restants (normalement des duplications exactes).


In [18]:
# --- 1) Code produit canonique par libellé Produit (le plus fréquent)
prod_canon = (
    dt_fbs_small
    .groupby("Produit")["Code Produit (FBS)"]
    .agg(lambda x: x.value_counts().idxmax())
)

dt_fbs_small = dt_fbs_small.merge(
    prod_canon.rename("code_produit_canon"),
    left_on="Produit",
    right_index=True,
    how="left"
)

before = dt_fbs_small.shape[0]
dt_fbs_small = dt_fbs_small[dt_fbs_small["Code Produit (FBS)"] == dt_fbs_small["code_produit_canon"]].copy()
dt_fbs_small.drop(columns=["code_produit_canon"], inplace=True)
after = dt_fbs_small.shape[0]
print(f"Filtre code produit canonique : {before:,} -> {after:,} lignes")

# --- 2) (Optionnel) Code zone canonique par M49, utile si tu as des agrégats type Monde/régions
zone_canon = (
    dt_fbs_small
    .groupby("Code zone (M49)")["Code zone"]
    .agg(lambda x: x.value_counts().idxmax())
)

dt_fbs_small = dt_fbs_small.merge(
    zone_canon.rename("code_zone_canon"),
    left_on="Code zone (M49)",
    right_index=True,
    how="left"
)

before = dt_fbs_small.shape[0]
dt_fbs_small = dt_fbs_small[dt_fbs_small["Code zone"] == dt_fbs_small["code_zone_canon"]].copy()
dt_fbs_small.drop(columns=["code_zone_canon"], inplace=True)
after = dt_fbs_small.shape[0]
print(f"Filtre code zone canonique : {before:,} -> {after:,} lignes")

# --- 3) Re-contrôle des doublons de pivot
pivot_keys = ["Code zone (M49)", "Zone", "Année", "Produit", "Élément"]
dup_n = dt_fbs_small.duplicated(subset=pivot_keys).sum()
print("Doublons restants sur (pays, année, produit, élément) :", dup_n)

# Si doublons restants : on regarde si c'est juste des duplications exactes (même Valeur)
if dup_n > 0:
    dups = dt_fbs_small[dt_fbs_small.duplicated(subset=pivot_keys, keep=False)].copy()
    check = (
        dups.groupby(pivot_keys)["Valeur"]
        .nunique()
        .sort_values(ascending=False)
        .head(20)
        .to_frame("nb_valeurs_distinctes")
    )
    display(check)

    # Si nb_valeurs_distinctes==1 partout => on peut drop_duplicates sans risque
    if (check["nb_valeurs_distinctes"] <= 1).all():
        dt_fbs_small = dt_fbs_small.drop_duplicates(subset=pivot_keys, keep="first").copy()
        print("✅ Doublons supprimés (valeurs identiques).")
    else:
        print("⚠️ Certains doublons ont des valeurs différentes -> on les inspecte avant de trancher.")
        display(dups.sort_values(pivot_keys).head(50))


Filtre code produit canonique : 12,791 -> 12,791 lignes
Filtre code zone canonique : 12,791 -> 12,791 lignes
Doublons restants sur (pays, année, produit, élément) : 0


## 16. Préparation des noms de colonnes (éléments) pour le format wide
On crée une version courte et standardisée de `Élément` pour obtenir des noms de colonnes lisibles après pivot.


In [19]:
def clean_colname(s: str) -> str:
    s = s.lower().strip()
    s = s.replace("é", "e").replace("è", "e").replace("ê", "e").replace("à", "a").replace("ç", "c").replace("ô", "o").replace("ù", "u").replace("î", "i").replace("ï", "i")
    s = s.replace("(", "").replace(")", "")
    s = s.replace("/", "_par_")
    s = s.replace(" - ", "_")
    s = s.replace(" ", "_")
    s = s.replace("__", "_")
    return s

dt_fbs_small["element_col"] = dt_fbs_small["Élément"].astype(str).apply(clean_colname)

# Vérif : collision de noms après nettoyage
collisions = (dt_fbs_small.groupby("element_col")["Élément"].nunique().sort_values(ascending=False))
collisions = collisions[collisions > 1]
if not collisions.empty:
    print("⚠️ Collisions : plusieurs Éléments donnent le même nom après nettoyage.")
    display(collisions.to_frame("nb_elements"))
    display(
        dt_fbs_small[dt_fbs_small["element_col"].isin(collisions.index)]
        [["Élément", "element_col"]]
        .drop_duplicates()
        .sort_values("element_col")
    )
else:
    print("✅ Pas de collisions sur les noms d'éléments.")


✅ Pas de collisions sur les noms d'éléments.


## 16.B Création des features et pivot en format wide
On combine `Produit` et `Élément` pour créer des variables uniques, puis on pivote pour obtenir une table “wide” : 1 ligne par pays et par année, 1 colonne par indicateur.


In [20]:
def clean_colname(s: str) -> str:
    s = str(s).lower().strip()
    s = (s.replace("é","e").replace("è","e").replace("ê","e")
           .replace("à","a").replace("ç","c").replace("ô","o")
           .replace("ù","u").replace("î","i").replace("ï","i"))
    s = s.replace("(", "").replace(")", "")
    s = s.replace("/", "_par_")
    s = s.replace(" - ", "_")
    s = s.replace(" ", "_")
    while "__" in s:
        s = s.replace("__", "_")
    return s

dt_fbs_small["prod_col"] = dt_fbs_small["Produit"].apply(clean_colname)
dt_fbs_small["elem_col"] = dt_fbs_small["Élément"].apply(clean_colname)
dt_fbs_small["feature"] = dt_fbs_small["prod_col"] + "__" + dt_fbs_small["elem_col"]

index_cols = ["Code zone (M49)", "Zone", "Année"]

# sécurité : vérifier unicité avant pivot
dup_feat = dt_fbs_small.duplicated(subset=index_cols + ["feature"]).sum()
print("Doublons sur (pays, année, feature) :", dup_feat)

dt_fbs_wide = (
    dt_fbs_small
    .pivot(index=index_cols, columns="feature", values="Valeur")
    .reset_index()
)

dt_fbs_wide.columns.name = None
print("dt_fbs_wide | shape =", dt_fbs_wide.shape)
display(dt_fbs_wide.head(10))


Doublons sur (pays, année, feature) : 0
dt_fbs_wide | shape = (1640, 11)


,Code zone (M49),Zone,Année,viande__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour,viande__exportations_quantite,viande__importations_quantite,viande__production,viande_de_volailles__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour,viande_de_volailles__exportations_quantite,viande_de_volailles__importations_quantite,viande_de_volailles__production
0,'001,Monde,2010,17.37,41925.00,38046.00,293012.00,6.30,14768.00,13175.00,99009.00
1,'001,Monde,2011,17.31,44096.00,40266.00,297369.00,6.47,15804.00,14242.00,102987.00
2,'001,Monde,2012,17.32,44388.00,40801.00,306050.00,6.50,16186.00,14445.00,106717.00
3,'001,Monde,2013,17.47,45911.00,41900.00,312994.00,6.62,16574.00,14837.00,110151.00
4,'001,Monde,2014,17.53,47009.00,42557.00,318549.00,6.63,16801.00,14383.00,113058.00
5,'001,Monde,2015,17.65,46901.00,42894.00,324272.00,6.75,16375.00,14296.00,116346.00
6,'001,Monde,2016,17.75,48313.00,44808.00,327965.00,6.88,17037.00,15120.00,120032.00
7,'001,Monde,2017,17.79,49974.00,45869.00,334218.00,6.95,17850.00,15418.00,124067.00
8,'002,Afrique,2010,7.57,204.00,1803.00,15825.00,2.60,78.00,1159.00,4712.00
9,'002,Afrique,2011,7.58,115.00,2023.00,16191.00,2.64,20.00,1332.00,4819.00


## 17. Contrôles post-pivot et extraction 2017
On vérifie les valeurs manquantes après pivot et on extrait l’année de référence (2017) pour préparer les jointures et le dataset final.


In [21]:
# 1) NA par colonne (top 20)
feature_cols = [c for c in dt_fbs_wide.columns if c not in index_cols]
na_pct = (dt_fbs_wide[feature_cols].isna().mean() * 100).sort_values(ascending=False)
display(na_pct.head(20).to_frame("NA_%"))

# 2) Extraire 2017 (version simple)
dt_fbs_2017 = dt_fbs_wide[dt_fbs_wide["Année"] == 2017].copy()
print("dt_fbs_2017 | shape =", dt_fbs_2017.shape)

# 3) Retirer l'agrégat Monde si présent (Code M49 = 001 d'après ton aperçu)
dt_fbs_2017 = dt_fbs_2017[dt_fbs_2017["Code zone (M49)"].astype(str).str.strip().ne("001")].copy()
print("dt_fbs_2017 sans Monde | shape =", dt_fbs_2017.shape)

display(dt_fbs_2017.head(10))


,NA_%
viande_de_volailles__exportations_quantite,13.60
viande__exportations_quantite,5.18
viande_de_volailles__production,0.98
viande_de_volailles__importations_quantite,0.30
viande__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour,0.00
viande__importations_quantite,0.00
viande__production,0.00
viande_de_volailles__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour,0.00


dt_fbs_2017 | shape = (205, 11)
dt_fbs_2017 sans Monde | shape = (205, 11)


,Code zone (M49),Zone,Année,viande__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour,viande__exportations_quantite,viande__importations_quantite,viande__production,viande_de_volailles__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour,viande_de_volailles__exportations_quantite,viande_de_volailles__importations_quantite,viande_de_volailles__production
7,'001,Monde,2017,17.79,49974.00,45869.00,334218.00,6.95,17850.00,15418.00,124067.00
15,'002,Afrique,2017,7.70,310.00,2792.00,19827.00,2.82,87.00,1855.00,6197.00
23,'004,Afghanistan,2017,4.25,0.00,39.00,293.00,0.81,NaN,29.00,28.00
31,'005,Amérique du Sud,2017,36.14,8969.00,1076.00,43743.00,18.39,4562.00,344.00,21512.00
39,'008,Albanie,2017,18.64,0.00,34.00,90.00,5.24,0.00,20.00,13.00
47,'009,Océanie,2017,40.29,2929.00,546.00,6431.00,16.13,66.00,95.00,1539.00
55,'011,Afrique occidentale,2017,4.91,6.00,550.00,3677.00,1.30,3.00,457.00,677.00
63,'012,Algérie,2017,8.88,1.00,68.00,809.00,3.13,0.00,2.00,284.00
71,'013,Amérique centrale,2017,26.15,637.00,2534.00,8613.00,14.06,22.00,1173.00,4304.00
79,'014,Afrique orientale,2017,5.32,43.00,118.00,4963.00,0.98,1.00,70.00,811.00


## 18. Vérification rapide des agrégats (Monde / régions)
On vérifie si la table 2017 contient des lignes non-pays (Monde, régions) afin de les exclure avant les analyses et jointures.


In [22]:
# Check explicite sur le libellé
display(dt_fbs_2017["Zone"].value_counts().head(20))

# Check sur motifs (si jamais)
mask_agg = dt_fbs_2017["Zone"].str.contains("monde|world|afrique|europe|asie|amerique|oceanie|union|income|income|develop", case=False, na=False)
print("Lignes suspectes (zones agrégées) :", mask_agg.sum())
display(dt_fbs_2017.loc[mask_agg, ["Code zone (M49)", "Zone"]].drop_duplicates().head(30))


Zone
Monde                     1
Nouvelle-Calédonie        1
Mexique                   1
Mongolie                  1
République de Moldova     1
Monténégro                1
Maroc                     1
Mozambique                1
Oman                      1
Namibie                   1
Népal                     1
Pays-Bas (Royaume des)    1
Vanuatu                   1
Mauritanie                1
Nouvelle-Zélande          1
Nicaragua                 1
Niger                     1
Nigéria                   1
Norvège                   1
Pakistan                  1
Name: count, dtype: int64

Lignes suspectes (zones agrégées) : 21


,Code zone (M49),Zone
7,'001,Monde
15,'002,Afrique
55,'011,Afrique occidentale
79,'014,Afrique orientale
87,'015,Afrique septentrionale
95,'017,Afrique centrale
103,'018,Afrique australe
151,'030,Asie orientale
175,'034,Asie méridionale
183,'035,Asie du Sud-Est


## 18bis. Suppression des agrégats (en gardant l’Afrique du Sud)
On retire les zones agrégées (Monde, continents, régions, UE…) qui ne sont pas des pays, tout en conservant l’Afrique du Sud qui est un vrai pays.


In [23]:
dt_fbs_2017_clean = dt_fbs_2017.copy()

# sécuriser le type et nettoyer
dt_fbs_2017_clean["Code zone (M49)"] = dt_fbs_2017_clean["Code zone (M49)"].astype(str).str.strip()
dt_fbs_2017_clean["Zone"] = dt_fbs_2017_clean["Zone"].astype(str).str.strip()

# Monde (M49=001) -> dehors quoi qu'il arrive
dt_fbs_2017_clean = dt_fbs_2017_clean[dt_fbs_2017_clean["Code zone (M49)"] != "001"].copy()

# détecter les agrégats via mots-clés (même logique que ton check)
mask_agg = dt_fbs_2017_clean["Zone"].str.contains(
    r"monde|world|afrique|europe|asie|amerique|oc[eé]anie|union|income|develop",
    case=False,
    na=False
)

# exception : Afrique du Sud doit rester
mask_sa = dt_fbs_2017_clean["Zone"].str.lower().eq("afrique du sud")

# on retire les agrégats sauf Afrique du Sud
to_drop = mask_agg & ~mask_sa

print("Lignes agrégées supprimées :", int(to_drop.sum()))
display(dt_fbs_2017_clean.loc[to_drop, ["Code zone (M49)", "Zone"]].drop_duplicates().sort_values(["Zone"]).head(50))

dt_fbs_2017_clean = dt_fbs_2017_clean[~to_drop].copy()

print("✅ Lignes restantes :", dt_fbs_2017_clean.shape[0])
print("✅ Nb de zones (pays) :", dt_fbs_2017_clean["Code zone (M49)"].nunique())


Lignes agrégées supprimées : 21


,Code zone (M49),Zone
15,'002,Afrique
103,'018,Afrique australe
95,'017,Afrique centrale
55,'011,Afrique occidentale
79,'014,Afrique orientale
87,'015,Afrique septentrionale
399,'142,Asie
407,'143,Asie centrale
183,'035,Asie du Sud-Est
175,'034,Asie méridionale


✅ Lignes restantes : 184
✅ Nb de zones (pays) : 184


## 19. Traitement simple des valeurs manquantes (imputation)
On remplit les NA pour rendre le dataset utilisable en ACP / clustering, avec une règle simple : flux/quantités → 0 ; variables “par personne” → médiane.


In [24]:
dt_fbs_2017_clean = dt_fbs_2017.copy()

index_cols = ["Code zone (M49)", "Zone", "Année"]
feature_cols = [c for c in dt_fbs_2017_clean.columns if c not in index_cols]

# Colonnes de flux/quantités (NA -> 0)
qty_cols = [
    c for c in feature_cols
    if (
        c.endswith("_exportations_quantite")
        or c.endswith("_importations_quantite")
        or c.endswith("_production")
        or c.endswith("_nourriture")
    )
]

# Colonnes "par personne" (NA -> médiane)
percap_cols = [
    c for c in feature_cols
    if (
        "kg_par_personne_par_jour" in c
        or "g_par_personne_par_jour" in c
        or "disponibilite" in c
        or "proteines" in c
    )
]

# (Sécurité) éviter double comptage
percap_cols = [c for c in percap_cols if c not in qty_cols]

# Imputation
dt_fbs_2017_clean[qty_cols] = dt_fbs_2017_clean[qty_cols].fillna(0)

for c in percap_cols:
    if dt_fbs_2017_clean[c].isna().any():
        dt_fbs_2017_clean[c] = dt_fbs_2017_clean[c].fillna(dt_fbs_2017_clean[c].median())

# Dernier filet : s'il reste des NA (cas rares), médiane
still_na = dt_fbs_2017_clean[feature_cols].isna().sum().sum()
print("NA restants après règles :", still_na)

if still_na > 0:
    for c in feature_cols:
        if dt_fbs_2017_clean[c].isna().any():
            dt_fbs_2017_clean[c] = dt_fbs_2017_clean[c].fillna(dt_fbs_2017_clean[c].median())

print("NA restants (final) :", dt_fbs_2017_clean[feature_cols].isna().sum().sum())

# Petit résumé
na_pct_after = (dt_fbs_2017_clean[feature_cols].isna().mean() * 100).sort_values(ascending=False)
display(na_pct_after.head(10).to_frame("NA_%"))


NA restants après règles : 0
NA restants (final) : 0


,NA_%
viande__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour,0.00
viande__exportations_quantite,0.00
viande__importations_quantite,0.00
viande__production,0.00
viande_de_volailles__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour,0.00
viande_de_volailles__exportations_quantite,0.00
viande_de_volailles__importations_quantite,0.00
viande_de_volailles__production,0.00


## 20. Exploration rapide — `dt_pop_2000_2018`
On inspecte le fichier population pour identifier les colonnes clés (codes pays, année, population) et préparer une jointure fiable avec `dt_fbs_2017_clean`.


In [25]:
df = dt_pop_2000_2018

print("dt_pop_2000_2018 | shape =", df.shape)
print("\nColonnes :")
print(list(df.columns))

display(df.head(10))
display(df.tail(10))
display(df.info())

# NA% + doublons
na_pct = (df.isna().mean() * 100).sort_values(ascending=False)
display(na_pct[na_pct > 0].head(20).to_frame("NA_%"))

print("Doublons (lignes identiques) :", df.duplicated().sum())


dt_pop_2000_2018 | shape = (4411, 15)

Colonnes :
['Code Domaine', 'Domaine', 'Code zone', 'Zone', 'Code Élément', 'Élément', 'Code Produit', 'Produit', 'Code année', 'Année', 'Unité', 'Valeur', 'Symbole', 'Description du Symbole', 'Note']


,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2000,2000,1000 personnes,20779.95,X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2001,2001,1000 personnes,21606.99,X,Sources internationales sûres,NaN
2,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2002,2002,1000 personnes,22600.77,X,Sources internationales sûres,NaN
3,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2003,2003,1000 personnes,23680.87,X,Sources internationales sûres,NaN
4,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2004,2004,1000 personnes,24726.68,X,Sources internationales sûres,NaN
5,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2005,2005,1000 personnes,25654.28,X,Sources internationales sûres,NaN
6,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2006,2006,1000 personnes,26433.05,X,Sources internationales sûres,NaN
7,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2007,2007,1000 personnes,27100.54,X,Sources internationales sûres,NaN
8,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2008,2008,1000 personnes,27722.28,X,Sources internationales sûres,NaN
9,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2009,2009,1000 personnes,28394.81,X,Sources internationales sûres,NaN


,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
4401,OA,Séries temporelles annuelles,181,Zimbabwe,511,Population totale,3010,Population-Estimations,2009,2009,1000 personnes,12526.97,X,Sources internationales sûres,NaN
4402,OA,Séries temporelles annuelles,181,Zimbabwe,511,Population totale,3010,Population-Estimations,2010,2010,1000 personnes,12697.72,X,Sources internationales sûres,NaN
4403,OA,Séries temporelles annuelles,181,Zimbabwe,511,Population totale,3010,Population-Estimations,2011,2011,1000 personnes,12894.32,X,Sources internationales sûres,NaN
4404,OA,Séries temporelles annuelles,181,Zimbabwe,511,Population totale,3010,Population-Estimations,2012,2012,1000 personnes,13115.15,X,Sources internationales sûres,NaN
4405,OA,Séries temporelles annuelles,181,Zimbabwe,511,Population totale,3010,Population-Estimations,2013,2013,1000 personnes,13350.37,X,Sources internationales sûres,NaN
4406,OA,Séries temporelles annuelles,181,Zimbabwe,511,Population totale,3010,Population-Estimations,2014,2014,1000 personnes,13586.71,X,Sources internationales sûres,NaN
4407,OA,Séries temporelles annuelles,181,Zimbabwe,511,Population totale,3010,Population-Estimations,2015,2015,1000 personnes,13814.63,X,Sources internationales sûres,NaN
4408,OA,Séries temporelles annuelles,181,Zimbabwe,511,Population totale,3010,Population-Estimations,2016,2016,1000 personnes,14030.33,X,Sources internationales sûres,NaN
4409,OA,Séries temporelles annuelles,181,Zimbabwe,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,14236.59,X,Sources internationales sûres,NaN
4410,OA,Séries temporelles annuelles,181,Zimbabwe,511,Population totale,3010,Population-Estimations,2018,2018,1000 personnes,14438.80,X,Sources internationales sûres,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4411 entries, 0 to 4410
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Code Domaine            4411 non-null   object 
 1   Domaine                 4411 non-null   object 
 2   Code zone               4411 non-null   int64  
 3   Zone                    4411 non-null   object 
 4   Code Élément            4411 non-null   int64  
 5   Élément                 4411 non-null   object 
 6   Code Produit            4411 non-null   int64  
 7   Produit                 4411 non-null   object 
 8   Code année              4411 non-null   int64  
 9   Année                   4411 non-null   int64  
 10  Unité                   4411 non-null   object 
 11  Valeur                  4411 non-null   float64
 12  Symbole                 4411 non-null   object 
 13  Description du Symbole  4411 non-null   object 
 14  Note                    258 non-null    

None

,NA_%
Note,94.15


Doublons (lignes identiques) : 0


## 21. Préparation de la table population 2017 et alignement du code M49
On extrait 2017 et on standardise le code pays (M49) pour filtrer uniquement les pays présents dans les données population.


In [26]:
# --- Adapter les noms de colonnes si besoin ---
# On essaye de deviner les colonnes usuelles
cols = dt_pop_2000_2018.columns

# candidates
m49_candidates = [c for c in cols if "m49" in c.lower()]
year_candidates = [c for c in cols if "ann" in c.lower() or "year" in c.lower()]
value_candidates = [c for c in cols if "val" in c.lower() or "pop" in c.lower()]

print("Candidats M49 :", m49_candidates)
print("Candidats Année :", year_candidates)
print("Candidats Valeur/Population :", value_candidates)

# ⚠️ Si les colonnes ne sont pas correctes, on les fixera manuellement juste après ce print.


Candidats M49 : []
Candidats Année : ['Code année', 'Année']
Candidats Valeur/Population : ['Valeur']


## 22. Préparation de la population 2017 (table de référence)
On extrait l’année 2017 et on nettoie le libellé de pays (`Zone`) afin de pouvoir filtrer les pays “valides” et joindre la population au dataset FBS.


In [27]:
# 1) extraire 2017
dt_pop_2017 = dt_pop_2000_2018[dt_pop_2000_2018["Année"] == 2017].copy()

# 2) nettoyage minimal
dt_pop_2017["Zone"] = dt_pop_2017["Zone"].astype(str).str.strip()

# 3) on garde uniquement les colonnes utiles
dt_pop_2017 = dt_pop_2017[["Code zone", "Zone", "Année", "Valeur", "Unité"]].copy()
dt_pop_2017.rename(columns={"Valeur": "population_2017", "Unité": "population_unite"}, inplace=True)

print("dt_pop_2017 | shape =", dt_pop_2017.shape)
display(dt_pop_2017.head(10))


dt_pop_2017 | shape = (236, 5)


,Code zone,Zone,Année,population_2017,population_unite
17,2,Afghanistan,2017,36296.11,1000 personnes
36,202,Afrique du Sud,2017,57009.76,1000 personnes
55,3,Albanie,2017,2884.17,1000 personnes
74,4,Algérie,2017,41389.19,1000 personnes
93,79,Allemagne,2017,82658.41,1000 personnes
112,6,Andorre,2017,77.00,1000 personnes
131,7,Angola,2017,29816.77,1000 personnes
150,258,Anguilla,2017,14.58,1000 personnes
169,8,Antigua-et-Barbuda,2017,95.43,1000 personnes
188,151,Antilles néerlandaises (ex),2017,275.19,1000 personnes


## 23. Filtrage “pays valides” et jointure population ↔ FBS (2017)
On conserve uniquement les pays présents dans le fichier population 2017, ce qui élimine naturellement la plupart des agrégats, puis on ajoute la population au dataset final 2017.


In [28]:
# Nettoyage côté FBS
dt_fbs_2017_final = dt_fbs_2017_clean.copy()
dt_fbs_2017_final["Zone"] = dt_fbs_2017_final["Zone"].astype(str).str.strip()

# 1) Filtrer aux pays présents dans la population 2017 (sur les noms)
valid_zones = set(dt_pop_2017["Zone"].unique())
before = dt_fbs_2017_final.shape[0]
dt_fbs_2017_final = dt_fbs_2017_final[dt_fbs_2017_final["Zone"].isin(valid_zones)].copy()
after = dt_fbs_2017_final.shape[0]
print(f"Filtre zones présentes dans population 2017 : {before} -> {after}")

# 2) Jointure population
dt_fbs_2017_final = dt_fbs_2017_final.merge(
    dt_pop_2017[["Zone", "population_2017", "population_unite"]],
    on="Zone",
    how="left"
)

# 3) Contrôle
missing_pop = dt_fbs_2017_final["population_2017"].isna().sum()
print("Population manquante après jointure :", missing_pop)

display(dt_fbs_2017_final.head(5))
print("dt_fbs_2017_final | shape =", dt_fbs_2017_final.shape)


Filtre zones présentes dans population 2017 : 205 -> 168
Population manquante après jointure : 0


,Code zone (M49),Zone,Année,viande__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour,viande__exportations_quantite,viande__importations_quantite,viande__production,viande_de_volailles__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour,viande_de_volailles__exportations_quantite,viande_de_volailles__importations_quantite,viande_de_volailles__production,population_2017,population_unite
0,'004,Afghanistan,2017,4.25,0.00,39.00,293.00,0.81,0.00,29.00,28.00,36296.11,1000 personnes
1,'008,Albanie,2017,18.64,0.00,34.00,90.00,5.24,0.00,20.00,13.00,2884.17,1000 personnes
2,'012,Algérie,2017,8.88,1.00,68.00,809.00,3.13,0.00,2.00,284.00,41389.19,1000 personnes
3,'024,Angola,2017,10.35,0.00,451.00,294.00,4.76,0.00,277.00,40.00,29816.77,1000 personnes
4,'028,Antigua-et-Barbuda,2017,40.35,0.00,11.00,0.00,31.50,0.00,7.00,0.00,95.43,1000 personnes


dt_fbs_2017_final | shape = (168, 13)


## 24. Exploration — `DisponibiliteAlimentaire_2017`
On explore ce fichier pour comprendre ses colonnes et vérifier s’il apporte des informations nouvelles par rapport aux variables déjà extraites du fichier FBS.


In [29]:
df = dt_dispo_2017

print("dt_dispo_2017 | shape =", df.shape)
print("\nColonnes :")
print(list(df.columns))

display(df.head(10))
display(df.tail(10))
display(df.info())

na_pct = (df.isna().mean() * 100).sort_values(ascending=False)
display(na_pct[na_pct > 0].head(20).to_frame("NA_%"))

print("Doublons (lignes identiques) :", df.duplicated().sum())


dt_dispo_2017 | shape = (176600, 14)

Colonnes :
['Code Domaine', 'Domaine', 'Code zone', 'Zone', 'Code Élément', 'Élément', 'Code Produit', 'Produit', 'Code année', 'Année', 'Unité', 'Valeur', 'Symbole', 'Description du Symbole']


,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2511,Blé et produits,2017,2017,Milliers de tonnes,4281.00,S,Données standardisées
1,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,2302.00,S,Données standardisées
2,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2511,Blé et produits,2017,2017,Milliers de tonnes,-119.00,S,Données standardisées
3,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5911,Exportations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,0.00,S,Données standardisées
4,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2511,Blé et produits,2017,2017,Milliers de tonnes,6701.00,S,Données standardisées
5,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5521,Aliments pour animaux,2511,Blé et produits,2017,2017,Milliers de tonnes,76.00,S,Données standardisées
6,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5527,Semences,2511,Blé et produits,2017,2017,Milliers de tonnes,344.00,S,Données standardisées
7,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5123,Pertes,2511,Blé et produits,2017,2017,Milliers de tonnes,642.00,S,Données standardisées
8,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5170,Résidus,2511,Blé et produits,2017,2017,Milliers de tonnes,0.00,S,Données standardisées
9,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5142,Nourriture,2511,Blé et produits,2017,2017,Milliers de tonnes,5640.00,S,Données standardisées


,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
176590,FBS,Nouveaux Bilans Alimentaire,181,Zimbabwe,5611,Importations - Quantité,2899,Miscellanees,2017,2017,Milliers de tonnes,20.00,S,Données standardisées
176591,FBS,Nouveaux Bilans Alimentaire,181,Zimbabwe,5911,Exportations - Quantité,2899,Miscellanees,2017,2017,Milliers de tonnes,1.00,S,Données standardisées
176592,FBS,Nouveaux Bilans Alimentaire,181,Zimbabwe,5301,Disponibilité intérieure,2899,Miscellanees,2017,2017,Milliers de tonnes,19.00,S,Données standardisées
176593,FBS,Nouveaux Bilans Alimentaire,181,Zimbabwe,5171,Alimentation pour touristes,2899,Miscellanees,2017,2017,Milliers de tonnes,0.00,S,Données standardisées
176594,FBS,Nouveaux Bilans Alimentaire,181,Zimbabwe,5170,Résidus,2899,Miscellanees,2017,2017,Milliers de tonnes,0.00,S,Données standardisées
176595,FBS,Nouveaux Bilans Alimentaire,181,Zimbabwe,5142,Nourriture,2899,Miscellanees,2017,2017,Milliers de tonnes,19.00,S,Données standardisées
176596,FBS,Nouveaux Bilans Alimentaire,181,Zimbabwe,645,Disponibilité alimentaire en quantité (kg/pers...,2899,Miscellanees,2017,2017,kg,1.33,Fc,Donnée calculée
176597,FBS,Nouveaux Bilans Alimentaire,181,Zimbabwe,664,Disponibilité alimentaire (Kcal/personne/jour),2899,Miscellanees,2017,2017,Kcal/personne/jour,1.00,Fc,Donnée calculée
176598,FBS,Nouveaux Bilans Alimentaire,181,Zimbabwe,674,Disponibilité de protéines en quantité (g/pers...,2899,Miscellanees,2017,2017,g/personne/jour,0.04,Fc,Donnée calculée
176599,FBS,Nouveaux Bilans Alimentaire,181,Zimbabwe,684,Disponibilité de matière grasse en quantité (g...,2899,Miscellanees,2017,2017,g/personne/jour,0.02,Fc,Donnée calculée


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176600 entries, 0 to 176599
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Code Domaine            176600 non-null  object 
 1   Domaine                 176600 non-null  object 
 2   Code zone               176600 non-null  int64  
 3   Zone                    176600 non-null  object 
 4   Code Élément            176600 non-null  int64  
 5   Élément                 176600 non-null  object 
 6   Code Produit            176600 non-null  int64  
 7   Produit                 176600 non-null  object 
 8   Code année              176600 non-null  int64  
 9   Année                   176600 non-null  int64  
 10  Unité                   176600 non-null  object 
 11  Valeur                  176600 non-null  float64
 12  Symbole                 176600 non-null  object 
 13  Description du Symbole  176600 non-null  object 
dtypes: float64(1), int64

None

,NA_%


Doublons (lignes identiques) : 0


## 25. Comparaison rapide des pays entre FBS final et Disponibilité 2017
On compare les listes de pays pour voir si le fichier de disponibilité couvre les mêmes pays, et si une jointure est simple.


In [30]:
# Standardiser les noms de zones pour comparer
fbs_zones = set(dt_fbs_2017_final["Zone"].astype(str).str.strip().unique())
dispo_zones = set(dt_dispo_2017["Zone"].astype(str).str.strip().unique()) if "Zone" in dt_dispo_2017.columns else set()

print("Nb pays FBS final :", len(fbs_zones))
print("Nb pays Dispo 2017 :", len(dispo_zones))

common = fbs_zones & dispo_zones
only_fbs = fbs_zones - dispo_zones
only_dispo = dispo_zones - fbs_zones

print("Commun :", len(common))
print("Uniquement FBS :", len(only_fbs))
print("Uniquement Dispo :", len(only_dispo))

# Afficher quelques exemples
print("\nExemples uniquement FBS :", sorted(list(only_fbs))[:20])
print("\nExemples uniquement Dispo :", sorted(list(only_dispo))[:20])


Nb pays FBS final : 168
Nb pays Dispo 2017 : 174
Commun : 162
Uniquement FBS : 6
Uniquement Dispo : 12

Exemples uniquement FBS : ['Comores', 'Libye', 'Papouasie-Nouvelle-Guinée', 'République arabe syrienne', 'République démocratique du Congo', 'Seychelles']

Exemples uniquement Dispo : ['Bermudes', 'Brunéi Darussalam', 'Bénin', 'Dominique', 'Japon', 'Mali', 'Pays-Bas', 'République centrafricaine', 'Soudan', 'Tchad', 'Togo', 'Turquie']


## 26. Identification des variables potentiellement redondantes
On repère dans `DisponibiliteAlimentaire_2017` les colonnes liées à la volaille, protéines, disponibilité alimentaire, etc., pour décider si on conserve ce fichier ou si on s’appuie uniquement sur FBS.


In [31]:
# Recherche de mots-clés dans les colonnes (adaptable)
keywords = ["vola", "oeuf", "protein", "protéin", "dispon", "kcal", "calorie", "viande", "poulet", "import", "export", "production"]
cols = dt_dispo_2017.columns.astype(str)

matched = []
for c in cols:
    cl = c.lower()
    if any(k in cl for k in keywords):
        matched.append(c)

print("Colonnes Dispo 2017 candidates (mots-clés) :")
display(pd.DataFrame({"colonne": matched}))


Colonnes Dispo 2017 candidates (mots-clés) :


,colonne


## 27. Décision sur `DisponibiliteAlimentaire_2017`
Ce fichier reprend la structure FBS (format long) pour l’année 2017. Comme nos variables 2017 proviennent déjà du fichier FBS normalisé (source unique + pipeline maîtrisé), on n’intègre pas ce fichier dans le dataset final ; il sert uniquement à la comparaison/validation et à justifier le choix de l’écarter.


In [32]:
# 1) Preuve rapide : structure FBS long-format
display(dt_dispo_2017[["Domaine","Élément","Produit","Année","Unité"]].head(10))

print("Années présentes :", sorted(dt_dispo_2017["Année"].unique())[:10], "...")

print("\nTop Produits (dispo_2017) :")
display(dt_dispo_2017["Produit"].value_counts().head(20))

print("\nTop Éléments (dispo_2017) :")
display(dt_dispo_2017["Élément"].value_counts().head(20))

# 2) Vérification que nos produits / éléments existent bien dedans (ils ne sont juste pas dans les *colonnes*)
prod_present = set(PRODUCTS_KEEP) & set(dt_dispo_2017["Produit"].unique())
elem_present = set(ELEMENTS_KEEP) & set(dt_dispo_2017["Élément"].unique())

print("Produits de notre sélection présents dans dispo_2017 :", sorted(list(prod_present)))
print("Éléments de notre sélection présents dans dispo_2017 :", sorted(list(elem_present)))


,Domaine,Élément,Produit,Année,Unité
0,Nouveaux Bilans Alimentaire,Production,Blé et produits,2017,Milliers de tonnes
1,Nouveaux Bilans Alimentaire,Importations - Quantité,Blé et produits,2017,Milliers de tonnes
2,Nouveaux Bilans Alimentaire,Variation de stock,Blé et produits,2017,Milliers de tonnes
3,Nouveaux Bilans Alimentaire,Exportations - Quantité,Blé et produits,2017,Milliers de tonnes
4,Nouveaux Bilans Alimentaire,Disponibilité intérieure,Blé et produits,2017,Milliers de tonnes
5,Nouveaux Bilans Alimentaire,Aliments pour animaux,Blé et produits,2017,Milliers de tonnes
6,Nouveaux Bilans Alimentaire,Semences,Blé et produits,2017,Milliers de tonnes
7,Nouveaux Bilans Alimentaire,Pertes,Blé et produits,2017,Milliers de tonnes
8,Nouveaux Bilans Alimentaire,Résidus,Blé et produits,2017,Milliers de tonnes
9,Nouveaux Bilans Alimentaire,Nourriture,Blé et produits,2017,Milliers de tonnes


Années présentes : [2017] ...

Top Produits (dispo_2017) :


Produit
Maïs et produits                   2593
Blé et produits                    2581
Pommes de Terre et produits        2486
Riz et produits                    2452
Lait - Excl Beurre                 2395
Oeufs                              2347
Légumineuses Autres et produits    2336
Orge et produits                   2301
Soja                               2222
Céréales, Autres                   2206
Légumes, Autres                    2204
Fruits, Autres                     2198
Arachides Decortiquees             2197
Edulcorants Autres                 2194
Haricots                           2141
Sucre Eq Brut                      2137
Graisses Animales Crue             2137
Feve de Cacao et produits          2088
Huil Plantes Oleif Autr            2083
Plantes Oleiferes, Autre           2077
Name: count, dtype: int64


Top Éléments (dispo_2017) :


Élément
Disponibilité intérieure                                         15905
Importations - Quantité                                          15260
Disponibilité alimentaire en quantité (kg/personne/an)           14618
Disponibilité de matière grasse en quantité (g/personne/jour)    14512
Disponibilité de protéines en quantité (g/personne/jour)         14507
Nourriture                                                       14498
Disponibilité alimentaire (Kcal/personne/jour)                   14476
Résidus                                                          12567
Exportations - Quantité                                          12113
Variation de stock                                               11299
Production                                                       10334
Pertes                                                            5813
Alimentation pour touristes                                       5560
Autres utilisations (non alimentaire)                             529

Produits de notre sélection présents dans dispo_2017 : ['Viande de Volailles']
Éléments de notre sélection présents dans dispo_2017 : ['Disponibilité de protéines en quantité (g/personne/jour)', 'Production']


## 28. Exploration — `dt_macro_norm` (indicateurs macro)
On explore la table macro pour identifier des indicateurs simples (ex : PIB/hab, inflation, etc.) utilisables en clustering, et vérifier la présence d’un identifiant pays exploitable (M49 ou Zone).


In [33]:
df = dt_macro_norm

print("dt_macro_norm | shape =", df.shape)
print("\nColonnes :")
print(list(df.columns))

display(df.head(10))
display(df.tail(10))
display(df.info())

na_pct = (df.isna().mean() * 100).sort_values(ascending=False)
display(na_pct[na_pct > 0].head(20).to_frame("NA_%"))

print("Doublons (lignes identiques) :", df.duplicated().sum())

# check rapide : combien d'éléments macro différents ?
if "Élément" in df.columns:
    print("\nNb d'indicateurs (Élément) :", df["Élément"].nunique())
    display(df["Élément"].value_counts().head(30))


dt_macro_norm | shape = (708973, 13)

Colonnes :
['Code zone', 'Code zone (M49)', 'Zone', 'Code Produit', 'Produit', 'Code Élément', 'Élément', 'Code année', 'Année', 'Unité', 'Valeur', 'Symbole', 'Note']


,Code zone,Code zone (M49),Zone,Code Produit,Produit,Code Élément,Élément,Code année,Année,Unité,Valeur,Symbole,Note
0,2,'004,Afghanistan,22008,Produit Intérieur Brut,6224,Valeur en devise locale standardisée,1970,1970,millions de MLS,78.70,X,NaN
1,2,'004,Afghanistan,22008,Produit Intérieur Brut,6224,Valeur en devise locale standardisée,1971,1971,millions de MLS,82.40,X,NaN
2,2,'004,Afghanistan,22008,Produit Intérieur Brut,6224,Valeur en devise locale standardisée,1972,1972,millions de MLS,71.80,X,NaN
3,2,'004,Afghanistan,22008,Produit Intérieur Brut,6224,Valeur en devise locale standardisée,1973,1973,millions de MLS,78.00,X,NaN
4,2,'004,Afghanistan,22008,Produit Intérieur Brut,6224,Valeur en devise locale standardisée,1974,1974,millions de MLS,97.00,X,NaN
5,2,'004,Afghanistan,22008,Produit Intérieur Brut,6224,Valeur en devise locale standardisée,1975,1975,millions de MLS,106.50,X,NaN
6,2,'004,Afghanistan,22008,Produit Intérieur Brut,6224,Valeur en devise locale standardisée,1976,1976,millions de MLS,115.00,X,NaN
7,2,'004,Afghanistan,22008,Produit Intérieur Brut,6224,Valeur en devise locale standardisée,1977,1977,millions de MLS,132.90,X,NaN
8,2,'004,Afghanistan,22008,Produit Intérieur Brut,6224,Valeur en devise locale standardisée,1978,1978,millions de MLS,148.49,X,NaN
9,2,'004,Afghanistan,22008,Produit Intérieur Brut,6224,Valeur en devise locale standardisée,1979,1979,millions de MLS,161.71,X,NaN


,Code zone,Code zone (M49),Zone,Code Produit,Produit,Code Élément,Élément,Code année,Année,Unité,Valeur,Symbole,Note
708963,5817,'902,Imp Nets Prod Alim Dvpm,22011,Revenu national brut,61290,Croissance annuelle US$ par habitant,2014,2014,%,5.21,X,NaN
708964,5817,'902,Imp Nets Prod Alim Dvpm,22011,Revenu national brut,61290,Croissance annuelle US$ par habitant,2015,2015,%,-2.86,X,NaN
708965,5817,'902,Imp Nets Prod Alim Dvpm,22011,Revenu national brut,61290,Croissance annuelle US$ par habitant,2016,2016,%,-3.01,X,NaN
708966,5817,'902,Imp Nets Prod Alim Dvpm,22011,Revenu national brut,61290,Croissance annuelle US$ par habitant,2017,2017,%,0.45,X,NaN
708967,5817,'902,Imp Nets Prod Alim Dvpm,22011,Revenu national brut,61290,Croissance annuelle US$ par habitant,2018,2018,%,-0.81,X,NaN
708968,5817,'902,Imp Nets Prod Alim Dvpm,22011,Revenu national brut,61290,Croissance annuelle US$ par habitant,2019,2019,%,-1.12,X,NaN
708969,5817,'902,Imp Nets Prod Alim Dvpm,22011,Revenu national brut,61290,Croissance annuelle US$ par habitant,2020,2020,%,-2.40,X,NaN
708970,5817,'902,Imp Nets Prod Alim Dvpm,22011,Revenu national brut,61290,Croissance annuelle US$ par habitant,2021,2021,%,9.47,X,NaN
708971,5817,'902,Imp Nets Prod Alim Dvpm,22011,Revenu national brut,61290,Croissance annuelle US$ par habitant,2022,2022,%,1.62,X,NaN
708972,5817,'902,Imp Nets Prod Alim Dvpm,22011,Revenu national brut,61290,Croissance annuelle US$ par habitant,2023,2023,%,-0.30,X,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 708973 entries, 0 to 708972
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Code zone        708973 non-null  int64  
 1   Code zone (M49)  708973 non-null  object 
 2   Zone             708973 non-null  object 
 3   Code Produit     708973 non-null  int64  
 4   Produit          708973 non-null  object 
 5   Code Élément     708973 non-null  int64  
 6   Élément          708973 non-null  object 
 7   Code année       708973 non-null  int64  
 8   Année            708973 non-null  int64  
 9   Unité            682131 non-null  object 
 10  Valeur           708973 non-null  float64
 11  Symbole          708973 non-null  object 
 12  Note             0 non-null       float64
dtypes: float64(2), int64(5), object(6)
memory usage: 70.3+ MB


None

,NA_%
Note,100.00
Unité,3.79


Doublons (lignes identiques) : 0

Nb d'indicateurs (Élément) : 20


Élément
Valeur US$                                                                                       67215
Croissance annuelle US$                                                                          65786
Valeur US$, aux prix de 2015                                                                     58283
Valeur en devise locale standardisée                                                             58135
Croissance annuelle US$, aux prix de 2015                                                        56891
Croissance annuelle en devise locale standardisée                                                56878
Valeur en devise locale standardisée, aux prix de 2015                                           50219
Croissance annuelle en devise locale standardisée, aux prix de 2015                              48996
Part du PIB US$                                                                                  40640
Part du PIB US$, aux prix de 2015                                

## 29. Sélection macro (E de PESTEL) — extraction 2017 sur le PIB
On extrait 2 variables économiques simples et robustes (PIB/hab et croissance du PIB/hab), sur le produit "Produit Intérieur Brut", puis on prépare une table prête à pivoter.


In [34]:
# --- Paramètres macro (simple)
MACRO_PRODUCT_KEEP = ["Produit Intérieur Brut"]

MACRO_ELEMENTS_KEEP = [
    "Valeur US$ par habitant, aux prix de 2015",
    "Croissance annuelle US$ par habitant, aux prix de 2015",
]

# --- Filtre 2017
dt_macro_2017_long = dt_macro_norm.copy()

# Standardiser M49 (string, zfill) pour jointure fiable
dt_macro_2017_long["Code zone (M49)"] = dt_macro_2017_long["Code zone (M49)"].astype(str).str.strip().str.zfill(3)

dt_macro_2017_long = dt_macro_2017_long[
    (dt_macro_2017_long["Année"] == 2017)
    & (dt_macro_2017_long["Produit"].isin(MACRO_PRODUCT_KEEP))
    & (dt_macro_2017_long["Élément"].isin(MACRO_ELEMENTS_KEEP))
].copy()

# Colonnes utiles uniquement
dt_macro_2017_long = dt_macro_2017_long[
    ["Code zone (M49)", "Zone", "Année", "Produit", "Élément", "Unité", "Valeur"]
].copy()

print("dt_macro_2017_long | shape =", dt_macro_2017_long.shape)
display(dt_macro_2017_long.head(10))

# Contrôle : unicité sur (M49, élément) attendu
dup_macro = dt_macro_2017_long.duplicated(subset=["Code zone (M49)", "Élément"]).sum()
print("Doublons sur (M49, Élément) :", dup_macro)


dt_macro_2017_long | shape = (488, 7)


,Code zone (M49),Zone,Année,Produit,Élément,Unité,Valeur
267,'004,Afghanistan,2017,Produit Intérieur Brut,"Valeur US$ par habitant, aux prix de 2015",USD,548.06
646,'004,Afghanistan,2017,Produit Intérieur Brut,"Croissance annuelle US$ par habitant, aux prix...",%,-1.06
3253,'710,Afrique du Sud,2017,Produit Intérieur Brut,"Valeur US$ par habitant, aux prix de 2015",USD,6121.74
3632,'710,Afrique du Sud,2017,Produit Intérieur Brut,"Croissance annuelle US$ par habitant, aux prix...",%,0.50
6803,'008,Albanie,2017,Produit Intérieur Brut,"Valeur US$ par habitant, aux prix de 2015",USD,4213.46
7182,'008,Albanie,2017,Produit Intérieur Brut,"Croissance annuelle US$ par habitant, aux prix...",%,3.79
10727,'012,Algérie,2017,Produit Intérieur Brut,"Valeur US$ par habitant, aux prix de 2015",USD,4162.15
11106,'012,Algérie,2017,Produit Intérieur Brut,"Croissance annuelle US$ par habitant, aux prix...",%,-0.74
13713,'276,Allemagne,2017,Produit Intérieur Brut,"Valeur US$ par habitant, aux prix de 2015",USD,43286.10
14092,'276,Allemagne,2017,Produit Intérieur Brut,"Croissance annuelle US$ par habitant, aux prix...",%,2.29


Doublons sur (M49, Élément) : 0


## 30. Pivot macro en format wide
On pivote les 2 indicateurs macro pour obtenir 1 ligne par pays (M49) et des colonnes prêtes à joindre au dataset FBS final.


In [35]:
def clean_colname(s: str) -> str:
    s = str(s).lower().strip()
    s = (s.replace("é","e").replace("è","e").replace("ê","e")
           .replace("à","a").replace("ç","c").replace("ô","o")
           .replace("ù","u").replace("î","i").replace("ï","i"))
    s = s.replace("(", "").replace(")", "")
    s = s.replace("/", "_par_").replace(" - ", "_").replace(" ", "_")
    while "__" in s:
        s = s.replace("__", "_")
    return s

dt_macro_2017_long["macro_feature"] = dt_macro_2017_long["Élément"].apply(clean_colname)
dt_macro_2017_long["macro_feature"] = "macro__" + dt_macro_2017_long["macro_feature"]

dt_macro_2017_wide = (
    dt_macro_2017_long
    .pivot_table(
        index=["Code zone (M49)", "Zone", "Année"],
        columns="macro_feature",
        values="Valeur",
        aggfunc="first"
    )
    .reset_index()
)

dt_macro_2017_wide.columns.name = None
print("dt_macro_2017_wide | shape =", dt_macro_2017_wide.shape)
display(dt_macro_2017_wide.head(10))


dt_macro_2017_wide | shape = (244, 5)


,Code zone (M49),Zone,Année,"macro__croissance_annuelle_us$_par_habitant,_aux_prix_de_2015","macro__valeur_us$_par_habitant,_aux_prix_de_2015"
0,'001,Monde,2017,2.23,10472.15
1,'002,Afrique,2017,0.99,1989.53
2,'004,Afghanistan,2017,-1.06,548.06
3,'005,Amérique du Sud,2017,-0.36,8780.53
4,'008,Albanie,2017,3.79,4213.46
5,'009,Océanie,2017,1.22,37114.37
6,'011,Afrique occidentale,2017,0.10,1843.49
7,'012,Algérie,2017,-0.74,4162.15
8,'013,Amérique centrale,2017,1.15,8808.91
9,'014,Afrique orientale,2017,3.06,888.17


## 31. Jointure FBS final ↔ Macro (sur Code zone (M49))
On enrichit le dataset FBS (168 pays) avec les indicateurs macro 2017 via une jointure sur le code M49 (plus robuste qu’une jointure sur les noms de pays).


In [36]:
dt_fbs_2017_enriched = dt_fbs_2017_final.copy()

# Standardiser M49 côté FBS aussi
dt_fbs_2017_enriched["Code zone (M49)"] = dt_fbs_2017_enriched["Code zone (M49)"].astype(str).str.strip().str.zfill(3)

before = dt_fbs_2017_enriched.shape[0]
dt_fbs_2017_enriched = dt_fbs_2017_enriched.merge(
    dt_macro_2017_wide.drop(columns=["Zone", "Année"], errors="ignore"),
    on="Code zone (M49)",
    how="left"
)
after = dt_fbs_2017_enriched.shape[0]

print(f"Lignes avant/après jointure : {before} -> {after}")

macro_cols = [c for c in dt_fbs_2017_enriched.columns if c.startswith("macro__")]
print("Colonnes macro ajoutées :", macro_cols)

missing_macro = dt_fbs_2017_enriched[macro_cols].isna().mean().sort_values(ascending=False) * 100
display(missing_macro.to_frame("NA_%"))


Lignes avant/après jointure : 168 -> 168
Colonnes macro ajoutées : ['macro__croissance_annuelle_us$_par_habitant,_aux_prix_de_2015', 'macro__valeur_us$_par_habitant,_aux_prix_de_2015']


,NA_%
"macro__croissance_annuelle_us$_par_habitant,_aux_prix_de_2015",0.60
"macro__valeur_us$_par_habitant,_aux_prix_de_2015",0.60


## 32. Imputation simple des variables macro
On remplit les valeurs macro manquantes par la médiane (faible impact et suffisant pour permettre ACP/k-means).


In [37]:
for c in macro_cols:
    if dt_fbs_2017_enriched[c].isna().any():
        dt_fbs_2017_enriched[c] = dt_fbs_2017_enriched[c].fillna(dt_fbs_2017_enriched[c].median())

print("NA restants sur macro :", int(dt_fbs_2017_enriched[macro_cols].isna().sum().sum()))
display(dt_fbs_2017_enriched.head(5))


NA restants sur macro : 0


,Code zone (M49),Zone,Année,viande__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour,viande__exportations_quantite,viande__importations_quantite,viande__production,viande_de_volailles__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour,viande_de_volailles__exportations_quantite,viande_de_volailles__importations_quantite,viande_de_volailles__production,population_2017,population_unite,"macro__croissance_annuelle_us$_par_habitant,_aux_prix_de_2015","macro__valeur_us$_par_habitant,_aux_prix_de_2015"
0,'004,Afghanistan,2017,4.25,0.00,39.00,293.00,0.81,0.00,29.00,28.00,36296.11,1000 personnes,-1.06,548.06
1,'008,Albanie,2017,18.64,0.00,34.00,90.00,5.24,0.00,20.00,13.00,2884.17,1000 personnes,3.79,4213.46
2,'012,Algérie,2017,8.88,1.00,68.00,809.00,3.13,0.00,2.00,284.00,41389.19,1000 personnes,-0.74,4162.15
3,'024,Angola,2017,10.35,0.00,451.00,294.00,4.76,0.00,277.00,40.00,29816.77,1000 personnes,-3.58,3739.38
4,'028,Antigua-et-Barbuda,2017,40.35,0.00,11.00,0.00,31.50,0.00,7.00,0.00,95.43,1000 personnes,1.88,16951.64


## 33. Finalisation du dataset (sélection des colonnes) et contrôles
On prépare un dataset propre pour l’ACP/clustering : on isole les identifiants, on garde uniquement des variables numériques, et on vérifie qu’on a bien au moins 100 pays.


In [38]:
dt_final = dt_fbs_2017_enriched.copy()

# Identifiants (à garder à part pour interprétation)
id_cols = ["Code zone (M49)", "Zone", "Année"]
id_cols = [c for c in id_cols if c in dt_final.columns]

print("Nb lignes (pays) :", dt_final.shape[0])
print("Nb pays uniques (M49) :", dt_final["Code zone (M49)"].nunique())

# Variables candidates = numériques uniquement (pour ACP/k-means)
num_cols = dt_final.select_dtypes(include=["int64", "int32", "float64", "float32"]).columns.tolist()
num_cols = [c for c in num_cols if c not in id_cols]

print("Nb variables numériques :", len(num_cols))
print("Exemples variables :", num_cols[:10])

# Contrôle NA final sur numériques
na_final = dt_final[num_cols].isna().mean().sort_values(ascending=False) * 100
display(na_final.head(15).to_frame("NA_%"))


Nb lignes (pays) : 168
Nb pays uniques (M49) : 168
Nb variables numériques : 11
Exemples variables : ['viande__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour', 'viande__exportations_quantite', 'viande__importations_quantite', 'viande__production', 'viande_de_volailles__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour', 'viande_de_volailles__exportations_quantite', 'viande_de_volailles__importations_quantite', 'viande_de_volailles__production', 'population_2017', 'macro__croissance_annuelle_us$_par_habitant,_aux_prix_de_2015']


,NA_%
viande__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour,0.00
viande__exportations_quantite,0.00
viande__importations_quantite,0.00
viande__production,0.00
viande_de_volailles__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour,0.00
viande_de_volailles__exportations_quantite,0.00
viande_de_volailles__importations_quantite,0.00
viande_de_volailles__production,0.00
population_2017,0.00
"macro__croissance_annuelle_us$_par_habitant,_aux_prix_de_2015",0.00


In [39]:
# Contrôle des éléments disponibles pour "Viande" vs les viandes détaillées
products_to_check = ["Viande", "Viande de Volailles", "Viande de Bovins", "Viande de porcins", "Viande d'Ovins/Caprins", "Viande, Autre"]

check = (
    dt_fbs_small[dt_fbs_small["Produit"].isin(products_to_check)]
    .groupby("Produit")["Élément"]
    .unique()
)

for p in products_to_check:
    if p in check.index:
        print(f"\n{p} -> nb éléments : {len(check[p])}")
        print(sorted(list(check[p]))[:30])
    else:
        print(f"\n{p} -> absent de dt_fbs_small")



Viande -> nb éléments : 4
['Disponibilité de protéines en quantité (g/personne/jour)', 'Exportations - quantité', 'Importations - quantité', 'Production']

Viande de Volailles -> nb éléments : 4
['Disponibilité de protéines en quantité (g/personne/jour)', 'Exportations - quantité', 'Importations - quantité', 'Production']

Viande de Bovins -> absent de dt_fbs_small

Viande de porcins -> absent de dt_fbs_small

Viande d'Ovins/Caprins -> absent de dt_fbs_small

Viande, Autre -> absent de dt_fbs_small


In [40]:
dt_simplified = dt_fbs_2017_enriched.copy()

# Préfixes à garder 
KEEP_PREFIXES = [
    "viande__",
    "viande_de_volailles__",
    "macro__",
]

id_cols = ["Code zone (M49)", "Zone", "Année"]
all_cols = dt_simplified.columns.tolist()

keep_cols = []
for c in all_cols:
    if c in id_cols:
        keep_cols.append(c)
    elif any(c.startswith(pref) for pref in KEEP_PREFIXES):
        keep_cols.append(c)
    elif c in ["population_2017"]:  
        keep_cols.append(c)

dt_simplified = dt_simplified[keep_cols].copy()

print("Avant :", dt_fbs_2017_enriched.shape, "| Après :", dt_simplified.shape)
display(dt_simplified.head(5))


Avant : (168, 15) | Après : (168, 14)


,Code zone (M49),Zone,Année,viande__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour,viande__exportations_quantite,viande__importations_quantite,viande__production,viande_de_volailles__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour,viande_de_volailles__exportations_quantite,viande_de_volailles__importations_quantite,viande_de_volailles__production,population_2017,"macro__croissance_annuelle_us$_par_habitant,_aux_prix_de_2015","macro__valeur_us$_par_habitant,_aux_prix_de_2015"
0,'004,Afghanistan,2017,4.25,0.00,39.00,293.00,0.81,0.00,29.00,28.00,36296.11,-1.06,548.06
1,'008,Albanie,2017,18.64,0.00,34.00,90.00,5.24,0.00,20.00,13.00,2884.17,3.79,4213.46
2,'012,Algérie,2017,8.88,1.00,68.00,809.00,3.13,0.00,2.00,284.00,41389.19,-0.74,4162.15
3,'024,Angola,2017,10.35,0.00,451.00,294.00,4.76,0.00,277.00,40.00,29816.77,-3.58,3739.38
4,'028,Antigua-et-Barbuda,2017,40.35,0.00,11.00,0.00,31.50,0.00,7.00,0.00,95.43,1.88,16951.64


In [41]:
# Si tu as créé dt_simplified avec KEEP
dt_final = dt_simplified.copy()


## 33bis. Recalcul des colonnes (après simplification)
On recalcule la liste des identifiants et des variables numériques à partir du dataset simplifié, pour éviter toute référence à des colonnes supprimées.


In [42]:
# Recalcul après simplification
id_cols = ["Code zone (M49)", "Zone", "Année"]
id_cols = [c for c in id_cols if c in dt_final.columns]

num_cols = dt_final.select_dtypes(include=["int64", "int32", "float64", "float32"]).columns.tolist()
num_cols = [c for c in num_cols if c not in id_cols]

print("dt_final | shape =", dt_final.shape)
print("Nb variables numériques (simplifié) :", len(num_cols))
print("Exemples :", num_cols[:10])


dt_final | shape = (168, 14)
Nb variables numériques (simplifié) : 11
Exemples : ['viande__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour', 'viande__exportations_quantite', 'viande__importations_quantite', 'viande__production', 'viande_de_volailles__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour', 'viande_de_volailles__exportations_quantite', 'viande_de_volailles__importations_quantite', 'viande_de_volailles__production', 'population_2017', 'macro__croissance_annuelle_us$_par_habitant,_aux_prix_de_2015']


## 33ter. Ajout de la stabilité politique (2017)
Pourquoi : ajouter une variable institutionnelle/risque pays, moins redondante avec les variables alimentaires, pour enrichir l’ACP/clustering


In [43]:
import re
import pycountry
import numpy as np

# --- 0) Base : repartir du fichier chargé (dt_pol_stab)
dt_pol_2017 = dt_pol_stab.copy()

# --- 1) Filtre 2017 + Total
dt_pol_2017 = dt_pol_2017[
    (dt_pol_2017["Year"] == 2017) &
    (dt_pol_2017["Granularity"].astype(str).str.strip() == "Total")
].copy()

# --- 2) Mapping Country -> M49 (string 3 digits)
OVERRIDES = {
    "Bolivia (Plurinational State of)": "068",
    "China, Hong Kong SAR": "344",
    "China, Macao SAR": "446",
    "China, mainland": "156",
    "China, Taiwan Province of": "158",
    "Democratic Republic of the Congo": "180",
    "Iran (Islamic Republic of)": "364",
    "Micronesia (Federated States of)": "583",
    "Turkey": "792",
    "Venezuela (Bolivarian Republic of)": "862",
}

def country_to_m49(name: str):
    name = str(name).strip()
    if name in OVERRIDES:
        return OVERRIDES[name]
    try:
        c = pycountry.countries.search_fuzzy(name)[0]
        return c.numeric  # ex "004"
    except Exception:
        return np.nan

dt_pol_2017["m49_key"] = dt_pol_2017["Country"].apply(country_to_m49)

# IMPORTANT : contrôler les non mappés AVANT de convertir en str
missing = dt_pol_2017["m49_key"].isna()
print("Pays non mappés (stabilité) :", dt_pol_2017.loc[missing, "Country"].unique()[:30])
print("Nb non mappés :", int(missing.sum()))

# garder seulement les mappés
dt_pol_2017 = dt_pol_2017[~missing].copy()

# standardiser clé
dt_pol_2017["m49_key"] = dt_pol_2017["m49_key"].astype(str).str.strip().str.replace("'", "", regex=False).str.zfill(3)

# --- 3) 1 ligne par pays (au cas où doublons)
dt_pol_2017 = (
    dt_pol_2017
    .groupby("m49_key", as_index=False)["Political_Stability"]
    .mean()
    .rename(columns={"Political_Stability": "political_stability_2017"})
)

# --- 4) Clé côté dt_final (enlever apostrophe et zfill)
def m49_clean(x):
    s = str(x).strip()
    s = s.replace("'", "")
    s = re.sub(r"\.0$", "", s)
    return s.zfill(3)

dt_final["m49_key"] = dt_final["Code zone (M49)"].apply(m49_clean)

# diagnostic match
match_rate = dt_final["m49_key"].isin(set(dt_pol_2017["m49_key"])).mean() * 100
print(f"Match keys (%) : {match_rate:.2f}%")

# --- 5) Merge + contrôle NA
dt_final = dt_final.drop(columns=["political_stability_2017"], errors="ignore")
dt_final = dt_final.merge(
    dt_pol_2017[["m49_key", "political_stability_2017"]],
    on="m49_key",
    how="left",
    validate="one_to_one"
)

na_pct = dt_final["political_stability_2017"].isna().mean() * 100
print(f"NA% political_stability_2017 après merge : {na_pct:.2f}%")

# si encore des NA : afficher quelques pays concernés
if dt_final["political_stability_2017"].isna().any():
    display(dt_final.loc[dt_final["political_stability_2017"].isna(), ["Code zone (M49)", "Zone"]].head(20))

# imputation médiane (si tu veux zéro NA)
dt_final["political_stability_2017"] = dt_final["political_stability_2017"].fillna(
    dt_final["political_stability_2017"].median()
)

# ménage
dt_final.drop(columns=["m49_key"], inplace=True)

# vérif finale
print("Stabilité présente ?", "political_stability_2017" in dt_final.columns)
display(dt_final[["Code zone (M49)", "Zone", "political_stability_2017"]].head(5))


Pays non mappés (stabilité) : []
Nb non mappés : 0
Match keys (%) : 97.62%
NA% political_stability_2017 après merge : 2.38%


,Code zone (M49),Zone
49,'258,Polynésie française
80,'410,République de Corée
108,'540,Nouvelle-Calédonie
112,'562,Niger


Stabilité présente ? True


,Code zone (M49),Zone,political_stability_2017
0,'004,Afghanistan,-2.80
1,'008,Albanie,0.38
2,'012,Algérie,-0.92
3,'024,Angola,-0.33
4,'028,Antigua-et-Barbuda,0.75


## 34. Export du dataset final (prêt pour ACP/clustering)
On exporte le dataset final nettoyé et enrichi en CSV pour l’utiliser dans le notebook ACP/clustering.


In [45]:
# Recalcul après ajout stabilité
id_cols = ["Code zone (M49)", "Zone", "Année"]
id_cols = [c for c in id_cols if c in dt_final.columns]

num_cols = dt_final.select_dtypes(include=["int64", "int32", "float64", "float32"]).columns.tolist()
num_cols = [c for c in num_cols if c not in id_cols]

dt_model = dt_final[id_cols + num_cols].copy()

out_path = "dataset_final_2017_fbs_macro_stab.csv"
dt_model.to_csv(out_path, index=False, encoding="utf-8")

print("Fichier exporté :", out_path)
print("Shape export :", dt_model.shape)
print("Stabilité dans l'export ?", "political_stability_2017" in dt_model.columns)

check = pd.read_csv(out_path, nrows=5)
print(check.columns.tolist())


Fichier exporté : dataset_final_2017_fbs_macro_stab.csv
Shape export : (168, 15)
Stabilité dans l'export ? True
['Code zone (M49)', 'Zone', 'Année', 'viande__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour', 'viande__exportations_quantite', 'viande__importations_quantite', 'viande__production', 'viande_de_volailles__disponibilite_de_proteines_en_quantite_g_par_personne_par_jour', 'viande_de_volailles__exportations_quantite', 'viande_de_volailles__importations_quantite', 'viande_de_volailles__production', 'population_2017', 'macro__croissance_annuelle_us$_par_habitant,_aux_prix_de_2015', 'macro__valeur_us$_par_habitant,_aux_prix_de_2015', 'political_stability_2017']


In [49]:
"political_stability_2017" in dt_model.columns, \
[c for c in dt_model.columns if "political_stability" in c]


(True, ['political_stability_2017'])